In [17]:
import pandas as pd
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes_edu = pd.read_csv('Sources/fiuba_1_postulantes_educacion_concat.csv')
postulantes_gen = pd.read_csv('Sources/fiuba_2_postulantes_genero_y_edad_concat.csv')

In [3]:
postulantes_edu.drop(['Unnamed: 0'],axis=1,inplace=True)
postulantes_edu.rename(columns={'nombre':'estudio'},inplace=True)
postulantes_edu.head()

,idpostulante,estudio,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [4]:
postulantes_gen.drop(['Unnamed: 0'],axis=1,inplace=True)
postulantes_gen.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [5]:
estudios = pd.get_dummies(postulantes_edu['estudio'])
for col in estudios.columns:
    estudios.loc[estudios.loc[:,col]==1,col] = postulantes_edu.loc[estudios.loc[:,col]==1,'estado']

In [6]:
estudios.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0,En Curso,0,0,0,0,0
1,0,0,0,En Curso,0,0,0
2,0,0,En Curso,0,0,0,0
3,0,En Curso,0,0,0,0,0
4,0,0,0,0,0,0,En Curso


In [7]:
postulantes_edu = postulantes_edu.merge(estudios, left_index=True, right_index=True)
postulantes_edu.drop(['estudio','estado'], axis=1, inplace=True)
postulantes_edu.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,ZjlZ,0,En Curso,0,0,0,0,0
1,NdJl,0,0,0,En Curso,0,0,0
2,5kNq,0,0,En Curso,0,0,0,0
3,8rYD,0,En Curso,0,0,0,0,0
4,1Wvj,0,0,0,0,0,0,En Curso


In [8]:
estados = ['','Abandonado', 'En Curso', 'Graduado']
for x in range(len(estados)):
    for col in postulantes_edu.columns:
        postulantes_edu.loc[postulantes_edu.loc[:,col]==estados[x],col] = x

In [9]:
groups = postulantes_edu.groupby('idpostulante')


In [10]:
df = pd.DataFrame()


In [11]:
import datetime
#df = pd.DataFrame()
df = []
i = 0
for indice in groups.indices:
    i = i + 1
    aux = groups.get_group(indice).max()
    aux = aux.tolist()
    df.append(aux)
df = pd.DataFrame.from_records(df)
df.head()


,0,1,2,3,4,5,6,7
0,0z5Dmrd,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,0z5JW1r,0.0,0.0,3.0,0.0,3.0,3.0,1.0
2,0z5VvGv,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,0zB01pE,0.0,0.0,0.0,0.0,0.0,3.0,1.0
4,0zB026d,0.0,0.0,0.0,0.0,3.0,3.0,0.0


In [12]:
for x in df.columns:
    nwName = postulantes_edu.columns[x]
    df.rename(columns={x:nwName}, inplace=True)

In [13]:
df = df.merge(postulantes_gen,on='idpostulante',how='outer')


In [14]:
df.to_csv('Sources/postulantes_proc.csv')


In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv')
postulantes.head()

,Unnamed: 0,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0,0z5Dmrd,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1965-10-22,MASC
1,1,0z5JW1r,0.0,0.0,3.0,0.0,3.0,3.0,1.0,1971-04-09,MASC
2,2,0z5VvGv,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1987-10-18,MASC
3,3,0zB01pE,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1989-12-01,MASC
4,4,0zB026d,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1978-02-23,FEM


In [12]:
def max_educacion(row):
    lista_de_niveles = ['Doctorado','Master','Posgrado','Universitario','Otro','Terciario/Técnico','Secundario']
    lista_de_indices = ['Secundario','Terciario/Técnico','Otro','Universitario','Posgrado','Master','Doctorado']
    lista_de_estados = [0,2, 1, 3]
    for nivel in lista_de_niveles:
        if row[nivel]==0: continue
        for estado in lista_de_estados:
            if row[nivel] == estado:
                nivel_def = lista_de_indices.index(nivel)
                return ((3*nivel_def) + estado)
    return -1
        


In [13]:
postulantes['nivel'] = postulantes.apply(lambda x: max_educacion(x), axis=1)

In [14]:
postulantes.head()

,Unnamed: 0,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,nivel
0,0,0z5Dmrd,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1965-10-22,MASC,12
1,1,0z5JW1r,0.0,0.0,3.0,0.0,3.0,3.0,1.0,1971-04-09,MASC,10
2,2,0z5VvGv,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1987-10-18,MASC,3
3,3,0zB01pE,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1989-12-01,MASC,10
4,4,0zB026d,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1978-02-23,FEM,6


In [15]:
def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        #date_new = int(date_new)
        return date_new
    except ValueError:
        return -1

In [21]:
fechasRDD = sc.parallelize(postulantes['fechanacimiento'])
postulantes['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [23]:
postulantes.drop(['Doctorado','Master','Posgrado','Universitario','Otro','Terciario/Técnico','Secundario'], axis=1, inplace=True)
postulantes.head()

,Unnamed: 0,idpostulante,fechanacimiento,sexo,nivel
0,0,0z5Dmrd,53.0,MASC,12
1,1,0z5JW1r,47.0,MASC,10
2,2,0z5VvGv,31.0,MASC,3
3,3,0zB01pE,29.0,MASC,10
4,4,0zB026d,40.0,FEM,6


In [24]:
postulantes.to_csv('Sources/postulantes_proc.csv')